<a href="https://colab.research.google.com/github/NerminHussein/HR-Analytics_Classifier/blob/main/HR_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **HR Analytics**








# **Problem Statement**

A random large company, employs at any given point of time, around 4000 employees. However, every year, around 15% of its employees leave the company and need to be replaced with the talent pool available in the job market. The management believes that this level of attrition (employees leaving, either on their own or because they got fired) is bad for the company, because of the following reasons -

The former employees’ projects get delayed, which makes it difficult to meet timelines, resulting in a reputation loss among consumers and partners
A sizeable department has to be maintained, for the purposes of recruiting new talent
More often than not, the new employees have to be trained for the job and/or given time to acclimatise themselves to the company
Hence, the management has contracted an HR analytics firm to understand what factors they should focus on, in order to curb attrition. In other words, they want to know what changes they should make to their workplace, in order to get most of their employees to stay. Also, they want to know which of these variables is most important and needs to be addressed right away.

Since you are one of the star analysts at the firm, this project has been given to you.

Goal of the case study
You are required to model the probability of attrition using a logistic regression. The results thus obtained will be used by the management to understand what changes they should make to their workplace, in order to get most of their employees to stay.




# Time Analysis

In [258]:
log_in=pd.read_csv('in_time.csv')
log_out=pd.read_csv('out_time.csv')
# Finding the columns where it contains only null values which indicates it being a day off for the company 
#Doing that for the log in Dataframe
for columnName,columnValue in log_in.iteritems():
  nu=columnValue.isnull().sum()
  if nu==len(log_in):
    log_in.drop([columnName],axis=1,inplace=True)
#Doing the same for logout Dataframe
for columnName,columnValue in log_out.iteritems():
  nu=columnValue.isnull().sum()
  if nu==len(log_out):
    log_out.drop([columnName],axis=1,inplace=True)
#Making sure the remaining null values are equal in both Dataframes
#This could indicate day off for the employees
nu_in=sum(log_in.isnull().sum())
nu_out=sum(log_out.isnull().sum())
print(nu_in==nu_out)

True


In [259]:
#Filling the null values with 0 to compre them later
log_in.replace(np.nan, 0)
log_out.replace(np.nan, 0)
#Comparing values from same column and index and if they match it will be replaced with null values
comp_df=log_in.compare(log_out,keep_shape=True)
#
nu_comp=(sum(comp_df.isnull().sum())/2)-len(log_in)
print(nu_comp==nu_in)


True


In [260]:
#Defining some parameters
work_hours_dict={}
g=1
#Iterating through each row in both dataframes at the same time
for (idx,s1),(_,s2) in zip(log_in.iterrows(),log_out.iterrows()):
  lis=[]
#Iterating through each value in that row
  for (id,v1),(_,v2) in zip(s1.iteritems(),s2.iteritems()):
#Dividing the time format into date and time and calculating time difference to assess hours worked
    a=str(v1).split()
    b=str(v2).split()
    if len(a)==2:
      c=datetime.strptime(a[1],'%H:%M:%S')
      d=datetime.strptime(b[1],'%H:%M:%S')
      tmdf=d-c
      tmdh=timedelta.total_seconds(tmdf)/3600
      lis.append(float(tmdh))
    else:
      lis.append('Day off')
#Creating a Dictionary with keys as employee ID and values a list of hours worked each day in order
  work_hours_dict[g]=lis
  g+=1
#Getting a new Dataframe from the Dictionary
work_hourst=pd.DataFrame.from_dict(work_hours_dict)


In [261]:
#The resulted dataframe is transposed so we transpose it back
work_hours=work_hourst.T
#An Extra column was added due to the substraction of employee id's from both dataframes so it has to be removed
work_hours.drop([0],axis=1,inplace=True)
#Adding an employee ID column to merge this data with the other dataframes
work_hours['Employee ID']=work_hours.index
emp=work_hours.pop('Employee ID')
work_hours.insert(0,'Employee ID',emp)
#Getting Column Names from original data
lis=[]
for columnName,columnValue in log_in.iteritems():
  lis.append(columnName)
work_hours.columns=lis
work_hours.rename(columns={'Unnamed: 0':'Employee ID'})

,Employee ID,2015-01-02,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-12,2015-01-13,2015-01-15,...,2015-12-17,2015-12-18,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-28,2015-12-29,2015-12-30,2015-12-31
1,1,7.208333,7.189722,7.410833,7.006667,7.289722,7.484444,7.262778,7.831111,7.346944,...,Day off,Day off,7.339167,7.395833,6.504722,7.596389,7.773889,7.315,7.778889,7.080278
2,2,8.109167,7.454722,Day off,7.396944,7.416667,7.150833,7.611389,7.278889,7.613056,...,7.928333,7.903056,7.753889,7.712222,7.435556,Day off,7.614722,7.9825,7.986111,8.227222
3,3,6.6925,7.265556,6.405278,6.765,7.345,6.861389,7.418611,6.999722,7.438333,...,7.218333,6.785833,7.163611,6.801667,6.730278,6.849722,7.023889,7.438889,7.538889,6.786389
4,4,7.338333,7.291944,6.943056,6.919444,6.850833,7.193056,6.998611,7.306389,6.876667,...,7.455833,7.629167,6.846667,7.326389,7.413611,7.085,7.447222,7.416667,7.366389,7.133056
5,5,8.055556,7.988056,7.6825,7.806111,7.662222,7.721667,8.365,8.257222,8.26,...,8.32,7.903611,7.665,7.9575,7.786944,8.249444,7.662222,8.268611,7.953333,8.018056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4406,4406,8.118056,8.840833,8.399444,8.845,8.223611,8.280833,8.360556,8.759167,8.355556,...,8.504722,8.083056,8.161389,8.2925,8.604444,8.596944,8.482222,9.074722,8.943611,8.508056
4407,4407,6.255556,Day off,5.393611,5.727778,6.193333,5.714444,6.052778,6.292222,6.013611,...,5.983611,5.925,5.830556,5.924722,6.110556,5.992778,5.669722,6.558333,5.702222,6.1475
4408,4408,7.276111,7.585278,7.641389,7.986111,7.296389,7.031667,7.775833,7.478333,8.122778,...,7.685278,7.785556,7.866111,7.753611,7.752222,7.401944,7.734444,Day off,7.760556,8.090278
4409,4409,9.525556,9.586944,9.788333,9.391111,9.204444,Day off,9.294167,9.599722,9.484444,...,9.255,9.969722,9.340833,9.3725,9.241667,9.697778,10.053056,9.012222,9.360556,9.616111


# Pre-processing

In [262]:
# Getting the model ready by importing the libraries that will be used
import pandas as pd
import numpy as np
import glob
import os
from datetime import datetime,timedelta

In [263]:
employee_survey_data=pd.read_csv('employee_survey_data.csv')
print(employee_survey_data.isnull().sum())


EmployeeID                  0
EnvironmentSatisfaction    25
JobSatisfaction            20
WorkLifeBalance            38
dtype: int64


In [264]:
# Summary of Data
print(employee_survey_data.describe())

        EmployeeID  EnvironmentSatisfaction  JobSatisfaction  WorkLifeBalance
count  4410.000000              4385.000000      4390.000000      4372.000000
mean   2205.500000                 2.723603         2.728246         2.761436
std    1273.201673                 1.092756         1.101253         0.706245
min       1.000000                 1.000000         1.000000         1.000000
25%    1103.250000                 2.000000         2.000000         2.000000
50%    2205.500000                 3.000000         3.000000         3.000000
75%    3307.750000                 4.000000         4.000000         3.000000
max    4410.000000                 4.000000         4.000000         4.000000


In [265]:
print(employee_survey_data.mean())
#mean                  2.723603         2.728246         2.761436
print(employee_survey_data.median())
#EnvironmentSatisfaction       3.0
#JobSatisfaction               3.0
#WorkLifeBalance               3.0

# Mean & Median Approximatery same so we choose Median to Fill the NAN Data

EmployeeID                 2205.500000
EnvironmentSatisfaction       2.723603
JobSatisfaction               2.728246
WorkLifeBalance               2.761436
dtype: float64
EmployeeID                 2205.5
EnvironmentSatisfaction       3.0
JobSatisfaction               3.0
WorkLifeBalance               3.0
dtype: float64


In [266]:
#Fill Null in column of EnvironmentSatisfaction with Median
employee_survey_data['EnvironmentSatisfaction'] = employee_survey_data['EnvironmentSatisfaction'].fillna(employee_survey_data['EnvironmentSatisfaction'].median())

In [267]:
#Fill Null in column of JobSatisfaction with Median
employee_survey_data['JobSatisfaction'] = employee_survey_data['JobSatisfaction'].fillna(employee_survey_data['JobSatisfaction'].median())

In [268]:
#Fill Null in column of WorkLifeBalance with Median
employee_survey_data['WorkLifeBalance'] = employee_survey_data['WorkLifeBalance'].fillna(employee_survey_data['WorkLifeBalance'].median())

In [269]:
#check if found NULL Values in employee_survey_data file 
print(employee_survey_data.isnull().sum())

EmployeeID                 0
EnvironmentSatisfaction    0
JobSatisfaction            0
WorkLifeBalance            0
dtype: int64


In [270]:
Manager_survey_data=pd.read_csv('manager_survey_data.csv')
print(Manager_survey_data.isnull().sum())

EmployeeID           0
JobInvolvement       0
PerformanceRating    0
dtype: int64


In [271]:
General_Data=pd.read_csv('general_data.csv')
print(General_Data.isnull().sum())

Age                         0
Attrition                   0
BusinessTravel              0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeID                  0
Gender                      0
JobLevel                    0
JobRole                     0
MaritalStatus               0
MonthlyIncome               0
NumCompaniesWorked         19
Over18                      0
PercentSalaryHike           0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           9
TrainingTimesLastYear       0
YearsAtCompany              0
YearsSinceLastPromotion     0
YearsWithCurrManager        0
dtype: int64


In [272]:
print(General_Data.mean())
#################################################
print("##########################")
print(General_Data.median())

Age                           36.923810
DistanceFromHome               9.192517
Education                      2.912925
EmployeeCount                  1.000000
EmployeeID                  2205.500000
JobLevel                       2.063946
MonthlyIncome              65029.312925
NumCompaniesWorked             2.694830
PercentSalaryHike             15.209524
StandardHours                  8.000000
StockOptionLevel               0.793878
TotalWorkingYears             11.279936
TrainingTimesLastYear          2.799320
YearsAtCompany                 7.008163
YearsSinceLastPromotion        2.187755
YearsWithCurrManager           4.123129
dtype: float64
##########################
Age                           36.0
DistanceFromHome               7.0
Education                      3.0
EmployeeCount                  1.0
EmployeeID                  2205.5
JobLevel                       2.0
MonthlyIncome              49190.0
NumCompaniesWorked             2.0
PercentSalaryHike             14.0
Sta

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  after removing the cwd from sys.path.


In [273]:
General_Data['NumCompaniesWorked'] = General_Data['NumCompaniesWorked'].fillna(General_Data['NumCompaniesWorked'].mean())

In [274]:
General_Data['TotalWorkingYears'] = General_Data['TotalWorkingYears'].fillna(General_Data['TotalWorkingYears'].mean())

In [275]:
print(General_Data.isnull().sum())

Age                        0
Attrition                  0
BusinessTravel             0
Department                 0
DistanceFromHome           0
Education                  0
EducationField             0
EmployeeCount              0
EmployeeID                 0
Gender                     0
JobLevel                   0
JobRole                    0
MaritalStatus              0
MonthlyIncome              0
NumCompaniesWorked         0
Over18                     0
PercentSalaryHike          0
StandardHours              0
StockOptionLevel           0
TotalWorkingYears          0
TrainingTimesLastYear      0
YearsAtCompany             0
YearsSinceLastPromotion    0
YearsWithCurrManager       0
dtype: int64


In [276]:
Merge_Employee_Manager=employee_survey_data.merge(Manager_survey_data, on='EmployeeID')

In [277]:
Merge_Employee_Manager


,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,1,3.0,4.0,2.0,3,3
1,2,3.0,2.0,4.0,2,4
2,3,2.0,2.0,1.0,3,3
3,4,4.0,4.0,3.0,2,3
4,5,4.0,1.0,3.0,3,3
...,...,...,...,...,...,...
4405,4406,4.0,1.0,3.0,3,3
4406,4407,4.0,4.0,3.0,2,3
4407,4408,1.0,3.0,3.0,3,4
4408,4409,4.0,1.0,3.0,2,3


In [278]:
print(Merge_Employee_Manager.isnull().sum())

EmployeeID                 0
EnvironmentSatisfaction    0
JobSatisfaction            0
WorkLifeBalance            0
JobInvolvement             0
PerformanceRating          0
dtype: int64


In [279]:
Merge_EM_Data=Merge_Employee_Manager.merge(General_Data, on='EmployeeID')

In [280]:
cnt=0
for i in Merge_EM_Data['Over18']:
  if i=='Y':
    cnt+=1
print(cnt)

# All the Employees OVer 18 Years so when we check it yes al the employee over 18 so we can drop it
Merge_EM_Data.drop('Over18',axis=1,inplace=True)


4410


In [281]:
Merge_EM_Data.head(5)


,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating,Age,Attrition,BusinessTravel,Department,...,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,1,3.0,4.0,2.0,3,3,51,No,Travel_Rarely,Sales,...,131160,1.0,11,8,0,1.0,6,1,0,0
1,2,3.0,2.0,4.0,2,4,31,Yes,Travel_Frequently,Research & Development,...,41890,0.0,23,8,1,6.0,3,5,1,4
2,3,2.0,2.0,1.0,3,3,32,No,Travel_Frequently,Research & Development,...,193280,1.0,15,8,3,5.0,2,5,0,3
3,4,4.0,4.0,3.0,2,3,38,No,Non-Travel,Research & Development,...,83210,3.0,11,8,3,13.0,5,8,7,5
4,5,4.0,1.0,3.0,3,3,32,No,Travel_Rarely,Research & Development,...,23420,4.0,12,8,2,9.0,2,6,0,4


In [282]:
print(Merge_EM_Data.isnull().sum())

EmployeeID                 0
EnvironmentSatisfaction    0
JobSatisfaction            0
WorkLifeBalance            0
JobInvolvement             0
PerformanceRating          0
Age                        0
Attrition                  0
BusinessTravel             0
Department                 0
DistanceFromHome           0
Education                  0
EducationField             0
EmployeeCount              0
Gender                     0
JobLevel                   0
JobRole                    0
MaritalStatus              0
MonthlyIncome              0
NumCompaniesWorked         0
PercentSalaryHike          0
StandardHours              0
StockOptionLevel           0
TotalWorkingYears          0
TrainingTimesLastYear      0
YearsAtCompany             0
YearsSinceLastPromotion    0
YearsWithCurrManager       0
dtype: int64


In [283]:
work_hours=work_hours.rename(columns={'Unnamed: 0':'EmployeeID'})
work_hours.head()

,EmployeeID,2015-01-02,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-12,2015-01-13,2015-01-15,...,2015-12-17,2015-12-18,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-28,2015-12-29,2015-12-30,2015-12-31
1,1,7.208333,7.189722,7.410833,7.006667,7.289722,7.484444,7.262778,7.831111,7.346944,...,Day off,Day off,7.339167,7.395833,6.504722,7.596389,7.773889,7.315,7.778889,7.080278
2,2,8.109167,7.454722,Day off,7.396944,7.416667,7.150833,7.611389,7.278889,7.613056,...,7.928333,7.903056,7.753889,7.712222,7.435556,Day off,7.614722,7.9825,7.986111,8.227222
3,3,6.6925,7.265556,6.405278,6.765,7.345,6.861389,7.418611,6.999722,7.438333,...,7.218333,6.785833,7.163611,6.801667,6.730278,6.849722,7.023889,7.438889,7.538889,6.786389
4,4,7.338333,7.291944,6.943056,6.919444,6.850833,7.193056,6.998611,7.306389,6.876667,...,7.455833,7.629167,6.846667,7.326389,7.413611,7.085,7.447222,7.416667,7.366389,7.133056
5,5,8.055556,7.988056,7.6825,7.806111,7.662222,7.721667,8.365,8.257222,8.26,...,8.32,7.903611,7.665,7.9575,7.786944,8.249444,7.662222,8.268611,7.953333,8.018056


In [284]:

print("Work_Hours",work_hours.shape)
print("All_Data",Merge_EM_Data.shape)

Work_Hours (4410, 250)
All_Data (4410, 28)


In [285]:
Merge_WH_ALLData=Merge_EM_Data.merge(work_hours, on='EmployeeID')
Merge_WH_ALLData.head()

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating,Age,Attrition,BusinessTravel,Department,...,2015-12-17,2015-12-18,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-28,2015-12-29,2015-12-30,2015-12-31
0,1,3.0,4.0,2.0,3,3,51,No,Travel_Rarely,Sales,...,Day off,Day off,7.339167,7.395833,6.504722,7.596389,7.773889,7.315,7.778889,7.080278
1,2,3.0,2.0,4.0,2,4,31,Yes,Travel_Frequently,Research & Development,...,7.928333,7.903056,7.753889,7.712222,7.435556,Day off,7.614722,7.9825,7.986111,8.227222
2,3,2.0,2.0,1.0,3,3,32,No,Travel_Frequently,Research & Development,...,7.218333,6.785833,7.163611,6.801667,6.730278,6.849722,7.023889,7.438889,7.538889,6.786389
3,4,4.0,4.0,3.0,2,3,38,No,Non-Travel,Research & Development,...,7.455833,7.629167,6.846667,7.326389,7.413611,7.085,7.447222,7.416667,7.366389,7.133056
4,5,4.0,1.0,3.0,3,3,32,No,Travel_Rarely,Research & Development,...,8.32,7.903611,7.665,7.9575,7.786944,8.249444,7.662222,8.268611,7.953333,8.018056
